# 자연어 처리를 위한 개념 정리
---

* ### 희소 표현(Sparse Representation)
 벡터의 특정 차원에 단어 혹은 의미를 __직접 매핑__하는 방식

* ### 분포 가설(distribution hypothesis)
유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다
>나는 밥을 먹는다.<br>
>나는 떡을 먹는다.<br>
>나는 _____을 먹는다.<br>

    위와 같이 **유사한 맥락**에서 나오는 단어들끼리는 두 단어 벡터 사이의 거리를 가깝게 하고, 그렇지 않은 단어는 멀어지도록 조정한다.


<br>

=> 따라서 분산 표현을 사용하면 희소 표현과 다르게 **단어 간의 유사도**를 계산할 수 있음<br><br>
###     <span style="color:red">Embedding 레이어가 바로 단어의 분산 표현을 구하기 위한 레이어.</span>

---

## 1. Embedding Layer 기초
 **'단어 n개를 사용하고 k차원으로 표현할 것'**이라고 전달하면 알아서 **분산 표현 사전을 구성**해 준다.
 ![Embedding Layer](embedding_layer.png)

### Weight는 자연스럽게 '단어의 개수', 'Embedding 사이즈'로 정의됨<br><br>

 * ### 또한 Embedding Layer는 원-핫 인코딩(One-Hot Encoding)과 결합하여 쓰임
"i i i i need some more coffee coffee coffee"란 문장을 원핫 인코딩을 적용해서 임베딩할 것.<br>
vocab(단어 사전)을 유의해서 볼 것

In [1]:
# One-Hot Encoding 이해를 위한 간단한 예제

import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

Metal device set to: Apple M2
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


2022-09-19 20:05:53.264018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 20:05:53.264143: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### 이 결과를 Linear Layer에 넣을 것

In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[-0.02925324 -0.66063195]
 [ 0.22809005 -0.67404604]
 [-0.01334894  0.19311988]
 [ 0.07958895 -0.14330125]
 [ 0.6256676   0.24200141]
 [ 0.42546546 -0.60430086]
 [-0.34569496 -0.66989154]
 [ 0.38482547  0.15331137]]

One-Hot Linear Result
[[-0.02925324 -0.66063195]
 [-0.02925324 -0.66063195]
 [-0.02925324 -0.66063195]
 [-0.02925324 -0.66063195]
 [ 0.22809005 -0.67404604]
 [-0.01334894  0.19311988]
 [ 0.07958895 -0.14330125]
 [ 0.6256676   0.24200141]
 [ 0.6256676   0.24200141]
 [ 0.6256676   0.24200141]]


=> 원-핫 벡터에 Linear 레이어를 적용하니 **Linear 레이어의 Weight에서 단어 인덱스 배열 [ 0, 0, 0, 0, 1, 2, 3, 4, 4, 4 ] 에 해당하는 행만 읽어옴**
 ![Embedding Layer](embedding_layer.png)
 #### 즉 그림에서 파란색 선 과정 =  각 단어를 원-핫 인코딩해서 Linear 연산하는 것을 의미함<br><br>



### + 주의점 : Embedding 레이어는 그저 단어를 대응 시켜 줄 뿐이니 미분이 불가능. 
    따라서 신경망 설계를 할 때, 어떤 연산 결과를 Embedding 레이어에 연결시킬 수 없다.
    Embedding Layer를 입력에 직접 연결되게 사용하는 방식으로 이용
 ---

## 2. Embedding Layer의 진행

In [4]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장으로 생각.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이며,
# 각 단어는 100차원으로 분산 표현 할 것.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


---

## 3. Recurrent Layer
문장이나 영상, 음성 등의 데이터는 한 장의 이미지 데이터와는 사뭇 다른 특성을 가짐. 바로 **순차적인(Sequential) 특성**
> 나는 밥을 [ ]는다.<br>
>> 앞에 나온 밥이라는 단어 때문에 '밥'이란 단어가 들어갈 것을 알 수 있음.


> 데이터의 나열 사이에 연관성이 없다고 해서 순차적인 데이터가 아니라고 할 수는 없음. [1, 2, 3, 오리, baby, 0.7] 라는 데이터도 요소들 간의 연관성이 없지만 시퀀스 데이터

###     <span style="color:red">이런 순차 데이터를 처리하기 위해 고안된 것이 바로 Recurrent Neural Network 또는 Recurrent 레이어(RNN)</span>
![RNN](RNN.png)


<br>

#### 하지만 RNN은 다음과 같은 문제점을 가지고 있음
![RNN_illustrated](RNN_problem.png)
* 첫 입력인 What의 정보가 마지막 입력인 ?에 다다라서는 **거의 희석된 모습**이 보임
### 이를 기울기 소실(Vanishing Gradient) 문제라고 함
---

## 4. RNN의 진행

In [6]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

RNN에 입력할 문장: What time is it ?


In [7]:
sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)


In [8]:
rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)


RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


>어떤 문장이 부정, 긍정인지 파악하는 것은 문장을 모두 읽은 후, 최종 Step의 Output만 확인해도 판단이 가능. <br>
>하지만 문장을 생성하는 경우라면 이전 단어를 입력으로 받아 생성된 모든 다음 단어, 즉 모든 Step에 대한 Output이 필요하다.


=> 이는 tf.keras.layers.SimpleRNN 레이어의 return_sequences 인자를 조절해서 해결이 가능하다.

In [9]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


#### 이것이 바로 기울기 소실(Vanishing Gradient)에 의해 장기 의존성(Long-Term Dependency)을 잘 다루지 못하는 문제를 해결하기 위해 등장한 LSTM


---
## 5. LSTM의 구조


![RNN&LSTM](RNN_LSTM.png)
<center>(좌측: RNN, 우측: LSTM)</center>


![LSTM](LSTM.png)
<center>(LSTM 구조 그래프)</center>


* $c_t$ : Cell state의 약자로 long-term memory 역할을 수행함
* Forget Gate Layer : cell state의 기존 정보를 얼마나 잊어버릴지를 결정하는 gate
* Input Gate Layer : 새롭게 들어온 정보를 기존 cell state에 얼마나 반영할지를 결정하는 gate
* Output Gate Layer : 새롭게 만들어진 cell state를 새로운 hidden state에 얼마나 반영할지를 결정하는 gate<br><br>


### 5-1. 변형 LSTM
* Gated Recurrent Units(GRU)
> [Gated Recurrent Units(GRU)](https://yjjo.tistory.com/18)


![GRU](GRU.png)
<center>(GRU 구조 그래프)</center>


LSTM은 GRU에 비해 Weight가 많기 때문에 충분한 데이터가 있는 상황에 적합하고, 반대로 GRU는 적은 데이터에도 웬만한 학습 성능을 보여줌<br><br>


---

## 6. 양방향(Bidirectional) RNN
> 날이 너무 <span style="color:orange">[?]</span> 에어컨을 켰다 


이런 예문이 있다면, 빈칸에 들어갈 말이 <span style="color:orange">'더워서'</span>인 것은 어렵지 않게 유추할 수 있음.<br>
하지만 **뒤에 나오는** '에어컨을 켰다'라는 말이 있기 때문에 유추가 가능했던 것으로, **순방향으로 진행되는 RNN은 이 정보를 모른 채로** 단어를 생성하게 됨.<br>
(날이 너무 추워서 에어컨을 켰다 같은 이상한 문장이 탄생할 수 있음)<br><br>


###     <span style="color:red">이런 문제를 해결하기 위해 고안된 것이 바로 양방향(Bidirectional) RNN</span>
(사실 단순히 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태이다.) : tf.keras.layers.Bidirectional()<br><br>


#### 사실 자연어 처리는 대체로 번역기를 만들 때 양방향(Bidirectional) RNN 계열의 네트워크, 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용함.

In [10]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

In [11]:
embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


#### 순방향 역방향 Weight를 각각 정의하므로 RNN의 2배 크기 Weight가 정의됨.
units를 64로 정의했고, 입력은 Embedding을 포함하여 (1, 5, 100), 그리고 양방향에 대한 Weight를 거쳐 나올 테니 출력은 (1, 5, 128)이 나오게 됨